In [ ]:
import pandas as pd
import numpy as np
import plotly_express as px
import hashlib

# Load datasets
data_athletes = pd.read_csv('Data/athlete_events.csv')
data_noc = pd.read_csv('Data/noc_regions.csv')

### Which countries are included in the dataset?

In [ ]:
# Extract unique region names
unique_regions = data_noc['region'].unique()
exclusions_region = ['Individual Olympic Athletes', 'NA'] # Exclusion list for regions

# TUV was described as 'NA' in the region column but had a name in the notes column
if 'Tuvalu' not in unique_regions:
    # Manually add 'Tuvalu' to the list of unique regions
    unique_regions = list(unique_regions) + ['Tuvalu']

# Print the list of unique region names
for region in unique_regions:
    if region not in exclusions_region:
        print(region)

### How many countries are included in the dataset?

In [ ]:
# Filter out excluded regions and count
filtered_regions = [
    region for region in unique_regions
        if region not in exclusions_region]

number_of_unique_regions = len(filtered_regions)

print(f"Total number of unique regions: {number_of_unique_regions}")

### Which sports are in the event?

In [ ]:
sports_list = data_athletes['Sport'].unique()

print("Types of sports:")
for sport in sports_list:
    print(sport)

### What types of medals are won?

In [ ]:
medal_types = data_athletes['Medal'].unique()

# Removes the rows where no medals has been won
medal_types_filtered = np.delete(medal_types, 0)

print("Types of medals:")
for medal in medal_types_filtered:
    print(medal)

### Sex distribiution chart

In [ ]:
# Remove duplicate entries based on 'ID' to ensure each athlete is counted only once.
data_unique_athletes = data_athletes.drop_duplicates(subset=['ID'])

# Count the number of male and female participants
sex_distribution = data_unique_athletes['Sex'].value_counts()

# Creating a pie chart
sex_distribution_piechart = px.pie(sex_distribution, 
            # The lambda function in the names argument is used to map "F" to "Female" and "M" to "Male".
             names=sex_distribution.index.map(lambda x: 'Female' if x == 'F' else 'Male'),
             values=sex_distribution.values,
             title='Sex distribution of all athletes',
             labels={'names' : 'Sex', "values" : 'Amount'})

sex_distribution_piechart.write_html(("../Projekt_OS_Australien/Visualisering/Sex_distribution_piechart.html"))

### Top 10 countries based on total medals won

In [ ]:
# Creates a new dataframe grouping that only shows the columns "Team" and "Total Medals"
# Value_counts Counts the values for each row
# Unstack transforms the groupby into a new dataframe
# fillna changes the missing data to having a value of 0 so it wont effect the counting
# Sum counts everthing in the first axis which is "Total Medals" after we used reset_index to both reset the index and change the name of the second column.
country_medals = data_athletes.groupby("NOC")["Medal"].value_counts().unstack().fillna(0).sum(axis=1).reset_index(name="Total Medals")

top_ten_countries = country_medals.sort_values(by="Total Medals", ascending=False).head(10)

top_ten_countries_diagram = px.bar(
    top_ten_countries,
    x="NOC", y="Total Medals",
    title="Top 10 countries based on total medals won:",
    color="NOC",
    labels={"NOC" : "Countries", "Total Medals" : "Medals"}
)

top_ten_countries_diagram.write_html("../Projekt_OS_Australien/Visualisering/Top_ten_countries_medals.html")

### Anonymization function for names in dataset

In [ ]:
#Anonymize a name using SHA-256 hashing algorithm
def anonymize_names(name):
    name_bytes = name.encode() # Transforming the string to bytes with .encode(), since the hash function demands it 
    hash_object = hashlib.sha256() # Creating an SHA-256 hashobject with hashlib.sha256()
    hash_object.update(name_bytes) # Updating the hashobject with the koded name-string (name_bytes)
    hashed_name = hash_object.hexdigest() # Making the hashe object into hexdecimal format
    return hashed_name

# Using the function to anonymize the name column
data_athletes['Name'] = data_athletes['Name'].apply(anonymize_names)